### 인스타그램 해시태그 이미지 크롤링

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
import time
import platform
import matplotlib.pyplot as plt
import sys
import re

### 인스타그램 접속 및 로그인

In [ ]:
driver = Chrome("c:/pydata/chromedriver.exe")
url =" https://www.instagram.com/"

uid = "id"
upw = "password"
driver.get(url)
time.sleep(2)
driver.find_element(By.NAME, "username").send_keys(uid)
driver.find_element(By.NAME, "password").send_keys(upw)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()

time.sleep(4)

keyword = "keyword"
url = "https://www.instagram.com/explore/tags/"+ quote(keyword) + "/"
driver.get(url)
time.sleep(5)

C:\Users\CJ\AppData\Local\Temp\ipykernel_22248\375969043.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("c:/pydata/chromedriver.exe")


In [ ]:
# 화면을 아래쪽으로 스크롤
def pang_Down():
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

In [ ]:
# 화면을 위쪽으로 스크롤
def pang_up():
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight - 1000)')

#### 이미지 src와 게시물 url 가져오기

In [ ]:
# 게시물 사진링크, 링크 가져오기
cnt = 0
img_src_href_lst = []
# href_lst = []
for i in range(10000):
    pang_Down()
    if cnt % 3 == 0:
            pang_up()
    if cnt == 5 :
        html = driver.page_source
        soup = bs(html, "html.parser")
        content = soup.find_all("div", "_aabd _aa8k _aanf") 
        for find in content:
            img_src_href_lst.append([find.find("img")["src"], find.find("a")["href"]])
            # href_lst.append(find.find("a")["href"])
        cnt = 1
    cnt += 1
    print(i+1)
    pang_Down()
    time.sleep(3)

len(img_src_href_lst)


In [ ]:
if not os.path.exists("./csv"):
    os.mkdir("csv")

In [ ]:
pd.DataFrame(img_src_href_lst).to_csv("./csv/"+ keyword +"_src_href.csv", index = False)

#### 가져온 사진 src로 저장

In [ ]:
# 결측지 제거

img_src_href_lst = pd.DataFrame(img_src_href_lst).dropna()
# 중복 제거
src = img_src_href_lst[0].drop_duplicates()
href = []
for s in src:
    for i in img_src_href_lst.values:
        if i[0] == s:
            href.append(i[1])
            break

In [ ]:
src.reset_index(inplace=True, drop=True)
src

In [ ]:
# 이미지 저장 폴더 생성
import os
PATH ="./이미지/"+keyword
if not os.path.exists("./이미지"):
    os.mkdir("./이미지")
if not os.path.exists(PATH):
    os.mkdir(PATH)
if not os.path.exists("./csv"):
    os.mkdir("csv")

In [ ]:
# 이미지 저장
from tqdm import tqdm
import requests 
from urllib.request import urlopen


for i in tqdm(range(len(src))):
    with urlopen(src[i]) as f:
        with open(PATH +'/6'+str(i+7781).zfill(6)+'.jpg', "wb") as h:
            img = f.read()
            h.write(img)

In [ ]:
# 이미지 이름하고 src, url dataframe 생성
img_id_url_src = pd.DataFrame({ "이미지id":os.listdir("./이미지/"+keyword+"/"),
                                 "url":href, 
                                "src":list(src.values.flatten())}) 
img_id_url_src.head(3)

In [ ]:
# dataframe 저장
img_id_url_src.to_csv("./csv/"+keyword+"_id_url_src.csv", index=False)
